<a href="https://colab.research.google.com/github/hope04302/freeSearch/blob/main/main_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 한국어 혐오 발언 탐지

## 필요 라이브러리 설치

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch==2.0.1 torchvision~=0.15.1 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-2.0-cp310-cp310-linux_x86_64.whl
!pip install torchaudio==2.0.1

!pip install transformers
!pip install datasets
!pip install torchmetrics
!pip install accelerate -U

# !pip install 'git+https://github.com/SKTBrain/KoBERT.git#egg=kobert_tokenizer&subdirectory=kobert_hf'
# !pip install sentencepiece

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.9/162.9 MB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.7/57.7 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 120.2/120.2 kB 5.5 MB/s eta 0:00:00
  Attempting uninstall: uritemplate
    Found existing installation: uritemplate 4.1.1
    Uninstalling uritemplate-4.1.1:
      Successfully uninstalled uritemplate-4.1.1
  Attempting uninstall: google-api-core
    Found existing installation: google-api-core 2.11.1
    Uninstalling google-api-core-2.11.1:
      Successfully uninstalled google-api-core-2.11.1
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 2.84.0
    Uninstalling google-api-python-client-2.84.0:
      Successfully uninstalled google-api-python-client-2.84.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conf

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.4/12.4 MB 87.4 MB/s eta 0:00:00
  Attempting uninstall: google-api-python-client
    Found existing installation: google-api-python-client 1.8.0
    Uninstalling google-api-python-client-1.8.0:
      Successfully uninstalled google-api-python-client-1.8.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cloud-tpu-client 0.10 requires google-api-python-client==1.8.0, but you have google-api-python-client 2.102.0 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.7/7.7 MB 15.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 30.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.8/3.8 MB 39.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 46.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 2.5 MB/s 

## 데이터셋 로드

- 학습, 검증, 테스트 데이터셋 준비
- 라벨 정보

      class_label:
        names:
          0: origin
          1: physical
          2: politics
          3: profanity
          4: age
          5: gender
          6: race
          7: religion
          8: not_hate_speech

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from datasets import load_dataset
from sklearn.preprocessing import MultiLabelBinarizer
from torchmetrics import Accuracy, F1Score, HammingDistance, AUROC, ExactMatch

from transformers import AutoTokenizer, AutoModel, TrainingArguments, Trainer, EarlyStoppingCallback

import torch_xla.core.xla_model as xm
import torch_xla.distributed.parallel_loader as pl
import torch_xla.distributed.xla_multiprocessing as xmp

# from kobert_tokenizer import KoBERTTokenizer

In [ ]:
NUM_LABELS = 9
BATCH_SIZE = 32
EPOCHS = 20
CORES = 1       # tpu vm = 1

# =========================================
# bert-base-multilingual-cased, monologg/koelectra-base-v3-discriminator, monologg/kobert, snunlp/KR-BERT-char16424
name_card = 'skt/kobert-base-v1'
model_name = "KoBERT/type_2-0"
model_path = f'/content/drive/MyDrive/models/{model_name}'
# =========================================

In [ ]:
device = xm.xla_device()

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(name_card, do_lower_case=False)
# tokenizer = KoBERTTokenizer.from_pretrained(name_card, do_lower_case=False)

The tokenizer class you load from this checkpoint is not the same type as the class this function is called from. It may result in unexpected tokenization. 
The tokenizer class you load from this checkpoint is 'XLNetTokenizer'. 
The class this function is called from is 'KoBERTTokenizer'.


In [ ]:
def tokenize_function(examples):

    model_input = tokenizer(examples['text'], max_length=128, truncation=True, padding="max_length")

    mlb = MultiLabelBinarizer(classes=[0,1,2,3,4,5,6,7,8])
    one_hot_labels = mlb.fit_transform(examples['label'])

    model_input['label'] = one_hot_labels
    # model_input['n_label'] = [sum(one_hot_label) for one_hot_label in one_hot_labels]

    return model_input

In [ ]:
dataset = load_dataset("jeanlee/kmhas_korean_hate_speech")

Generating train split:   0%|          | 0/78977 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/8776 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/21939 [00:00<?, ? examples/s]

In [ ]:
train, valid, test = dataset.map(tokenize_function, batched=True).values()

Map:   0%|          | 0/78977 [00:00<?, ? examples/s]

Map:   0%|          | 0/8776 [00:00<?, ? examples/s]

Map:   0%|          | 0/21939 [00:00<?, ? examples/s]

## 모델 로드

In [ ]:
# ===============================================

# class FocalLoss(nn.Module):

#     def __init__(self, alpha=0.25, gamma=2):

#         super(FocalLoss, self).__init__()
#         self.alpha = alpha
#         self.gamma = gamma


#     def forward(self, inputs, targets):

#         BCE_loss = F.binary_cross_entropy_with_logits(inputs, targets, reduction='none')
#         pt = torch.exp(-BCE_loss)

#         F_loss = self.alpha * (1 - pt) ** self.gamma * BCE_loss
#         return F_loss.mean()


class Model(nn.Module):

    def __init__(self):

        super(Model, self).__init__()
        self.model = AutoModel.from_pretrained(name_card)

        self.classifier = nn.Linear(self.model.config.hidden_size, NUM_LABELS)

        self.loss = nn.BCEWithLogitsLoss()
        # self.loss = FocalLoss(alpha=0.25, gamma=2)


    def forward(self, input_ids, attention_mask, token_type_ids=None, labels=None):

        model_input= {
            'input_ids': input_ids,
            'attention_mask': attention_mask,
        }

        if token_type_ids is not None:
            model_input['token_type_ids'] = token_type_ids

        hidden = self.model(**model_input)

        cls = hidden.last_hidden_state[:, 0, :]
        hidden = hidden.last_hidden_state[:, 1:, :]

        logits = self.classifier(cls + hidden.mean(dim=1))

        loss = self.loss(logits, labels)

        model_output = {
            'loss': loss,
            'logits': logits
        }

        return model_output

# ===============================================

## 성능 지표 계산

In [ ]:
def compute_metrics(eval_pred):

    # ==================================================

    threshold = 0.5

    logits, labels = eval_pred

    logits = torch.Tensor(logits)
    labels = torch.Tensor(labels).long()

    sigmoid = torch.nn.Sigmoid()
    probs = sigmoid(torch.Tensor(logits))

    preds = torch.zeros(size = probs.size())
    preds[probs >= threshold] = 1

    n_preds = preds.sum(dim=1)
    n_labels = labels.sum(dim=1)

    # ==================================================

    accuracy = Accuracy(task='multilabel', num_labels=NUM_LABELS)
    f1_macro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='macro')
    f1_micro = F1Score(task="multilabel", num_labels=NUM_LABELS, average='micro')
    f1_weight = F1Score(task="multilabel", num_labels=NUM_LABELS, average='weighted')
    em = ExactMatch(task='multiclass', num_classes=2)
    auroc = AUROC(task='multilabel', num_labels=NUM_LABELS, average='micro')
    hamming = HammingDistance(task="multiclass", num_classes=2)
    n_int_accuracy = Accuracy(task='multiclass', num_classes=NUM_LABELS)

    each_f1_score = F1Score(task='multilabel', num_labels=NUM_LABELS, average=None)

    print(each_f1_score(preds, labels))

    metrics = {'accuracy': accuracy(preds, labels),
               'f1_macro': f1_macro(preds, labels),
               'f1_micro': f1_micro(preds, labels),
               'f1_weighted': f1_weight(preds, labels),
               'auroc': auroc(preds, labels),
               'hamming_loss': hamming(preds, labels),
               'em': em(preds, labels),
               'n_int_accuracy': n_int_accuracy(n_preds, n_labels)}

    return metrics

## 모델 학습

In [ ]:
training_args = TrainingArguments(

    output_dir=model_path,
    do_train=True,
    do_eval=True,
    evaluation_strategy="epoch",
    num_train_epochs=EPOCHS,
    per_device_train_batch_size=BATCH_SIZE,
    per_device_eval_batch_size=BATCH_SIZE,
    logging_strategy="epoch",
    logging_dir='./logs',
    learning_rate=2e-5,
    run_name="v1",
    seed=42,
    tpu_num_cores=CORES,

    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model='eval_f1_micro',
    save_total_limit=1,
    # label_names=['labels', 'n_label'],

)

In [ ]:
def _mp_train_fn(index=None, frags=None):

    model = Model().to(device)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=valid,
        compute_metrics=compute_metrics,
        # callbacks=[EarlyStoppingCallback(early_stopping_patience=3)],
    )

    trainer.train()
    trainer.save_model()

In [ ]:
xmp.spawn(_mp_train_fn, args=(), nprocs=CORES, start_method='fork')

Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.008900,0.007364,0.954358,0.732000,0.817662,0.820004,0.894440,0.045642,0.748861,0.875114,67.738400,129.912000,4.060000
2,0.005900,0.006406,0.963157,0.754365,0.853784,0.852422,0.917287,0.036843,0.801162,0.903487,33.635200,261.631000,8.176000
3,0.004700,0.007679,0.957865,0.756737,0.834345,0.836418,0.909300,0.042135,0.770624,0.887420,33.264100,264.549000,8.267000
4,0.003500,0.007467,0.964702,0.764072,0.859348,0.857763,0.919077,0.035298,0.815520,0.916591,33.066900,266.127000,8.316000
5,0.002500,0.008850,0.963626,0.783431,0.855838,0.856743,0.918936,0.036374,0.808455,0.912944,32.690000,269.195000,8.412000
6,0.001800,0.010321,0.963512,0.755381,0.855292,0.855587,0.918396,0.036488,0.808569,0.914654,33.450900,263.072000,8.221000
7,0.001400,0.011858,0.964423,0.767378,0.858338,0.856728,0.918745,0.035577,0.816203,0.922288,33.526100,262.482000,8.203000


tensor([0.7765, 0.7755, 0.8238, 0.8374, 0.7888, 0.6980, 0.2222, 0.8148, 0.8511])
tensor([0.7813, 0.7992, 0.8351, 0.8624, 0.8122, 0.7237, 0.2564, 0.8214, 0.8975])
tensor([0.7691, 0.7960, 0.8352, 0.8210, 0.8207, 0.7552, 0.3030, 0.8380, 0.8724])
tensor([0.7932, 0.8056, 0.8399, 0.8621, 0.8083, 0.7384, 0.2778, 0.8491, 0.9024])
tensor([0.7972, 0.8027, 0.8332, 0.8673, 0.8293, 0.7423, 0.4348, 0.8478, 0.8963])
tensor([0.7864, 0.8115, 0.8355, 0.8649, 0.8232, 0.7434, 0.2051, 0.8316, 0.8967])
tensor([0.7908, 0.8092, 0.8259, 0.8599, 0.8179, 0.7383, 0.3125, 0.8500, 0.9020])


Epoch,Training Loss,Validation Loss,Accuracy,F1 Macro,F1 Micro,F1 Weighted,Auroc,Hamming Loss,Em,N Int Accuracy,Runtime,Samples Per Second,Steps Per Second
1,0.008900,0.007364,0.954358,0.732000,0.817662,0.820004,0.894440,0.045642,0.748861,0.875114,67.738400,129.912000,4.060000
2,0.005900,0.006406,0.963157,0.754365,0.853784,0.852422,0.917287,0.036843,0.801162,0.903487,33.635200,261.631000,8.176000
3,0.004700,0.007679,0.957865,0.756737,0.834345,0.836418,0.909300,0.042135,0.770624,0.887420,33.264100,264.549000,8.267000
4,0.003500,0.007467,0.964702,0.764072,0.859348,0.857763,0.919077,0.035298,0.815520,0.916591,33.066900,266.127000,8.316000
5,0.002500,0.008850,0.963626,0.783431,0.855838,0.856743,0.918936,0.036374,0.808455,0.912944,32.690000,269.195000,8.412000
6,0.001800,0.010321,0.963512,0.755381,0.855292,0.855587,0.918396,0.036488,0.808569,0.914654,33.450900,263.072000,8.221000
7,0.001400,0.011858,0.964423,0.767378,0.858338,0.856728,0.918745,0.035577,0.816203,0.922288,33.526100,262.482000,8.203000
8,0.001100,0.013851,0.962980,0.773063,0.852144,0.849198,0.914166,0.037020,0.812557,0.923314,33.555400,262.253000,8.195000
9,0.000900,0.013320,0.963942,0.771412,0.856654,0.856294,0.918340,0.036058,0.810962,0.916363,33.556000,262.248000,8.195000
10,0.000700,0.014550,0.964031,0.783017,0.857071,0.856200,0.918736,0.035969,0.813127,0.919667,37.160600,236.810000,7.400000


tensor([0.7683, 0.7802, 0.8319, 0.8617, 0.8042, 0.7224, 0.4444, 0.8485, 0.8959])
tensor([0.7855, 0.8162, 0.8362, 0.8573, 0.8300, 0.7368, 0.3429, 0.8392, 0.8987])
tensor([0.7753, 0.7983, 0.8269, 0.8657, 0.8298, 0.7482, 0.4706, 0.8322, 0.9003])
tensor([0.7842, 0.8080, 0.8383, 0.8567, 0.8294, 0.7471, 0.4762, 0.8603, 0.8998])
tensor([0.7914, 0.8120, 0.8377, 0.8620, 0.8312, 0.7352, 0.4878, 0.8417, 0.8992])
tensor([0.7934, 0.7947, 0.8380, 0.8545, 0.8146, 0.7475, 0.4000, 0.8500, 0.9009])
tensor([0.7922, 0.8000, 0.8315, 0.8646, 0.8271, 0.7485, 0.4103, 0.8529, 0.9012])
tensor([0.7947, 0.7968, 0.8375, 0.8644, 0.8309, 0.7498, 0.3000, 0.8403, 0.9016])
tensor([0.7910, 0.8078, 0.8358, 0.8603, 0.8300, 0.7419, 0.3000, 0.8462, 0.9003])
tensor([0.7987, 0.8088, 0.8437, 0.8642, 0.8253, 0.7527, 0.3590, 0.8432, 0.9016])
tensor([0.8040, 0.8139, 0.8395, 0.8698, 0.8260, 0.7463, 0.3333, 0.8432, 0.9020])
tensor([0.8005, 0.8103, 0.8413, 0.8666, 0.8290, 0.7502, 0.3333, 0.8531, 0.9024])
tensor([0.8000, 0.8111, 0.84

## 모델 예측

In [ ]:
def _mp_test_fn(index=None, frags=None):

    state_dict = torch.load(f"/{model_path}/pytorch_model.bin")
    model = Model().to(device)
    model.load_state_dict(state_dict=state_dict)

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train,
        eval_dataset=test,
        compute_metrics=compute_metrics,
    )

    print(trainer.evaluate())

In [ ]:
xmp.spawn(_mp_test_fn, args=(), nprocs=CORES, start_method='fork')

tensor([0.8127, 0.7976, 0.8486, 0.8824, 0.8235, 0.7335, 0.4660, 0.8440, 0.9004])
{'eval_loss': 0.019394265487790108, 'eval_accuracy': 0.9649026989936829, 'eval_f1_macro': 0.7898732423782349, 'eval_f1_micro': 0.8614000082015991, 'eval_f1_weighted': 0.8610188961029053, 'eval_auroc': 0.9201797246932983, 'eval_hamming_loss': 0.03509730100631714, 'eval_em': 0.8163544535636902, 'eval_n_int_accuracy': 0.9193673133850098, 'eval_runtime': 160.9268, 'eval_samples_per_second': 136.41, 'eval_steps_per_second': 4.263}
